In [20]:
import pickle
from sklearn.model_selection import train_test_split
import os
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization 
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical

In [23]:
root = '/mnt/c/Users/weiwya/Desktop/icon_data_encoded/'
files = []
for (dirpath, dirnames, filenames) in os.walk(root):
    files.extend(filenames)
    break



In [36]:
train_data, tests_data = {}, {}
for f in files:
    try:
        d = pickle.load(open("%s%s" %(root, f), 'rb' ))
        k = f[:-2]
        train, test = train_test_split(d, test_size=0.2, random_state=1234)
        train_data[k] = train
        tests_data[k] = test
    except:
        print(f)
#     print(k, len(train), len(test), len(d))
neg_train = train_data['_negative']
neg_test = tests_data['_negative']
del train_data['_negative']
del tests_data['_negative']
class_names = np.array(list(train_data.keys()))
class_name_to_idx = {n:i for i, n in enumerate(class_names)}
idx_to_class_name = {v:k for k, v in class_name_to_idx.items()}


In [125]:
def get_data(data_dict, classes, idx_to_class_name, return_size=None, neg_samples=None):
    data = []
    labels = []
    counts = 0

    for c in classes:
        class_name = idx_to_class_name[c]
        d = data_dict[class_name]
        size = d.shape[0]
        if return_size is None or size<return_size:
            data.append(d)
            counts += size
            labels += [c] * size
        else:
            data.append(d[:return_size])
            counts += return_size
            labels += [c]* return_size  
    
    
    if neg_samples is not None:
        idx = np.random.choice(neg_samples.shape[0], neg_samples.shape[0],replace=False)
        if return_size is not None:
            data.append(neg_samples[idx[:return_size]])
            labels+= [-1] * return_size
            counts += return_size

        else:
            data.append(neg_samples[idx])
            labels += [-1] * neg_samples.shape[0]
            counts += neg_samples.shape[0]
            
        #add other classes as neg examples as well
        other = []
        for k, v in data_dict.items():
            if k not in classes:
                other.append(v)
        other = np.vstack(other)
        sample_idx = np.random.choice(other.shape[0],  other.shape[0],  replace=False)
        if return_size is not None:
            other = other[sample_idx[:return_size]]
            labels += [-1] * return_size
            counts += return_size
        else:
            #TODO!!!! set better size here
            other = other[sample_idx[:1000]]
            labels += [-1] * 1000
            counts += 1000
            
        data.append(other)
    
            
    #shuffle data for good measure
    data = np.vstack(data)
    labels = np.array(labels)

    idx = np.random.choice(np.arange(counts), size=counts, replace=False)
    return data[idx], np.array(labels)[idx]

In [86]:
#basic MLP 
#TODO:Swap out for more  sophisticated models 

def fit_basic_model(train_data, train_labels, epochs=50, verbose=False):
    input_dim = train_data.shape[1]
    n_classes = len(np.unique(train_labels))
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(input_dim, )))
    model.add(Dropout(.25))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

    model.fit(train_data, to_categorical(train_labels), 
                        epochs = epochs, 
                        verbose = verbose,
                        validation_split = 0.25,
                        shuffle=True)
    return model

    
def get_accuracy(predict, acutual):
    return np.sum(predict==acutual)/len(acutual)


def gen_task_classes(total_classes, n_tasks=10, shuffle=False):
    class_labels = np.arange(total_classes)
    if shuffle:
        np.random.shuffle(class_labels)
    return np.array_split(class_labels, n_tasks)

In [126]:
def fit_all_task_models(train_dict, idx_to_class_name, neg_samples, n_tasks=10, n_sample_per_task=1000, 
                        n_label_repeats=1, shuffle_labels=True,  verbose=False):
    
    total_labels = len(train_dict)
    print('total_labels %s' %total_labels)
    print('n_tasks!!! %s' %n_tasks)

    labels_for_tasks = []
    for i in range(n_label_repeats):
        labels_for_tasks.append(gen_task_classes(total_labels, n_tasks, shuffle=shuffle_labels))


    labels_for_tasks = np.vstack(labels_for_tasks)
    total_tasks = len(labels_for_tasks)
    print("total number of tasks %s" %total_tasks)
                         
    transformers = {}
    task_data = {}
    task_label_lookup = {}
    voters = {}
    #this part tranins transformers for all tasks
    for task in range(total_tasks):
        actual_label_to_task_labels = {}
        task_labels_to_actual_labels = {} 
        
        task_labels = labels_for_tasks[task]        
        data, label = get_data(train_dict, task_labels, idx_to_class_name, return_size=n_sample_per_task,
                               neg_samples=neg_samples)
        
        task_labels = np.append(task_labels, -1) 
        task_labels = np.sort(task_labels) 
        
        for i, l in enumerate(task_labels):
            actual_label_to_task_labels[l] = i
            task_labels_to_actual_labels[i] = l

        label = [actual_label_to_task_labels[x] for x in label]
        
        model = fit_basic_model(data, label, verbose=verbose)
       
        #store for later stage
        transformers[task] = model
        task_label_lookup[task] = task_labels_to_actual_labels
        task_data[task] = (data,label)
                
        print('done_transformer %s' %task)
    print('training deciders')
    
    #this part trains voter, using transformers previously trained
    for task in range(total_tasks):
        d, l = task_data[task]
        transformed = []
        #partition current task data using all  models 
        for m in transformers.values():
            mm = Model(inputs=m.inputs, outputs=m.layers[-2].output)
            transformed.append(mm.predict(d))
        transformed = np.hstack(transformed)
        voter = fit_basic_model(transformed, l, verbose=verbose)
        voters[task] = voter
        print('done_voter %s' %task)
    
    return transformers, voters, task_label_lookup

In [127]:
#This is the decider part of task-unaware, 
#currenly it does argmax of sums
#TODO: exp w argmax of argmax, 
def vote_on_data(data, transforms, voters, task_label_lookup, total_classes=105):
    n_tasks = len(voters)
    n_classes = total_classes
    n_data = len(data)

    transformed = []
    for n in range(n_tasks):
        t = transformers[n]
        m = Model(inputs=t.inputs, outputs=t.layers[-2].output)
        transformed.append(m.predict(data))
    transformed  = np.hstack(transformed)


    all_prob = np.zeros((n_data, total_classes+1))

    for n in range(n_tasks):
        label = task_label_lookup[n]
        v = voters[n]
        vv = v.predict(transformed)
        for i, p in enumerate(vv):
            for j, pp in enumerate(p):
                all_prob[i][label[j]] +=  pp
    
    all_prob = all_prob[:, :-1]
    return all_prob.argmax(axis=1)

def get_task_prediction(data, transformers, voters, task, task_label_lookup):
    t = transformers[task]
    prd = t.predict(data).argmax(axis=1) 
    actual = []
    for p in prd:
        actual.append(task_label_lookup[task][p])
    return np.array(actual)

In [140]:
transformers, voters, task_label_lookup = \
        fit_all_task_models(train_data, idx_to_class_name, n_tasks=7, 
                            n_label_repeats=10, n_sample_per_task=1000, neg_samples=neg_train)

total_labels 105
n_tasks!!! 7
total number of tasks 70


/home/weiwya/essex-graphs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:35: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


done_transformer 0
done_transformer 1
done_transformer 2
done_transformer 3
done_transformer 4
done_transformer 5
done_transformer 6
done_transformer 7
done_transformer 8
done_transformer 9
done_transformer 10
done_transformer 11
done_transformer 12
done_transformer 13
done_transformer 14
done_transformer 15
done_transformer 16
done_transformer 17
done_transformer 18
done_transformer 19
done_transformer 20
done_transformer 21
done_transformer 22
done_transformer 23
done_transformer 24
done_transformer 25
done_transformer 26
done_transformer 27
done_transformer 28
done_transformer 29
done_transformer 30
done_transformer 31
done_transformer 32
done_transformer 33
done_transformer 34
done_transformer 35
done_transformer 36
done_transformer 37
done_transformer 38
done_transformer 39
done_transformer 40
done_transformer 41
done_transformer 42
done_transformer 43
done_transformer 44
done_transformer 45
done_transformer 46
done_transformer 47
done_transformer 48
done_transformer 49
done_trans

In [141]:
#check how many times an label occurred in all tasks
from collections import defaultdict
occur = defaultdict(int)
for v in task_label_lookup.values():
    for vv in v.values():
        occur[vv] += 1
        
total_correct = 0
total = 0
total_classes = len(train_data)
for label, data in tests_data.items(): 
    predict = vote_on_data(data, transformers, voters, task_label_lookup,total_classes=total_classes)
    c_class = class_name_to_idx[label]
    total_correct += np.sum( predict == [c_class]*len(data)) 
    total += data.shape[0]
    print(label,  occur[c_class], get_accuracy(predict, [c_class]*len(data)))

print(total_correct / total)

add 10 0.5988372093023255
airplane 10 0.609375
alarm 10 0.23711340206185566
arrow_down 10 0.43155452436194897
arrow_left 10 0.22151898734177214
arrow_right 10 0.18012422360248448
arrow_up 10 0.5397727272727273
attach 10 0.0
bag 10 0.1588785046728972
barcode 10 0.7171717171717171
battery 10 0.435
bluetooth 10 0.6125461254612546
bookmark 10 0.5910543130990416
brightness 10 0.40336134453781514
calculator 10 0.652452025586354
calendar 10 0.528
call 10 0.5024630541871922
camera 10 0.7029096477794793
car 10 0.6612111292962357
cart 10 0.6723484848484849
chart 10 0.28346456692913385
check_mark 10 0.603448275862069
clock 10 0.6303501945525292
close 10 0.5798319327731093
cloud 10 0.6882591093117408
computer 10 0.4691358024691358
contrast 10 0.453125
credit_card 10 0.559610705596107
crop 10 0.44360902255639095
cursor 10 0.43333333333333335
cut 10 0.6336898395721925
dashboard 10 0.49074074074074076
delete 10 0.23404255319148937
dollar 10 0.47468354430379744
download 10 0.09574468085106383
edit 10 

In [142]:
class_to_task_look_up = defaultdict(list)
for task, v in task_label_lookup.items():
    for vv in v.values():
        class_to_task_look_up[vv].append(task)

total_classes = len(train_data)
total_correct, total =0,0
for label, data in tests_data.items(): 
    c_class = class_name_to_idx[label]
    tasks = class_to_task_look_up[c_class]
    correct = 0
    for task in tasks:
        #TODO, need to average prob
        predict = get_task_prediction(data, transformers, voters, task, task_label_lookup)
        correct += np.sum( predict == [c_class]*len(data))
    correct /= len(tasks)
    total_correct += correct
    
    total += data.shape[0]
    print(label,  occur[c_class], correct/data.shape[0])

print(total_correct / total)
    

add 10 0.6773255813953488
airplane 10 0.783984375
alarm 10 0.5407216494845362
arrow_down 10 0.6761020881670533
arrow_left 10 0.6775316455696202
arrow_right 10 0.49316770186335407
arrow_up 10 0.6778409090909091
attach 10 0.0
bag 10 0.33551401869158876
barcode 10 0.7747474747474747
battery 10 0.6502500000000001
bluetooth 10 0.7468634686346863
bookmark 10 0.7335463258785943
brightness 10 0.5109243697478991
calculator 10 0.8439232409381663
calendar 10 0.7847999999999999
call 10 0.6709359605911329
camera 10 0.7946401225114854
car 10 0.7944353518821603
cart 10 0.7886363636363636
chart 10 0.5167322834645669
check_mark 10 0.725
clock 10 0.8182879377431908
close 10 0.7468487394957983
cloud 10 0.8155870445344129
computer 10 0.6920987654320988
contrast 10 0.56875
credit_card 10 0.7197080291970803
crop 10 0.5894736842105264
cursor 10 0.6693333333333333
cut 10 0.7970588235294118
dashboard 10 0.6893518518518519
delete 10 0.5340425531914893
dollar 10 0.6645569620253164
download 10 0.33936170212765954

In [156]:
labels_for_tasks = gen_task_classes(len(train_data), 7, shuffle=True)
data, labels = [], []
for i, classes in enumerate(labels_for_tasks):
    d, l = get_data(train_data, classes, idx_to_class_name, return_size=1000, neg_samples=None)
    data.append(d)
    labels.append(l)#[i] * d.shape[0]
data = np.vstack(data)
labels = np.hstack(labels)
idx = np.random.choice(data.shape[0], data.shape[0], replace=False)
data = data[idx]
labels = labels[idx]
print(data.shape, labels.shape)
model = fit_basic_model(data, labels, verbose=True, epochs=200)  


(81261, 1000) (81261,)
Epoch 1/200
1905/1905 [==============================] - 4s 2ms/step - loss: 3.9086 - accuracy: 0.1385 - val_loss: 3.4878 - val_accuracy: 0.2123
Epoch 2/200
1905/1905 [==============================] - 4s 2ms/step - loss: 3.3559 - accuracy: 0.2307 - val_loss: 3.2200 - val_accuracy: 0.2655
Epoch 3/200
1905/1905 [==============================] - 4s 2ms/step - loss: 3.1543 - accuracy: 0.2662 - val_loss: 3.0730 - val_accuracy: 0.2941
Epoch 4/200
1905/1905 [==============================] - 4s 2ms/step - loss: 3.0191 - accuracy: 0.2938 - val_loss: 2.9699 - val_accuracy: 0.3106
Epoch 5/200
1905/1905 [==============================] - 4s 2ms/step - loss: 2.9195 - accuracy: 0.3126 - val_loss: 2.8874 - val_accuracy: 0.3330
Epoch 6/200
1905/1905 [==============================] - 4s 2ms/step - loss: 2.8371 - accuracy: 0.3313 - val_loss: 2.8265 - val_accuracy: 0.3468
Epoch 7/200
1905/1905 [==============================] - 4s 2ms/step - loss: 2.7670 - accuracy: 0.3462 - va

1905/1905 [==============================] - 4s 2ms/step - loss: 1.8404 - accuracy: 0.5416 - val_loss: 2.2398 - val_accuracy: 0.5097
Epoch 58/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.8313 - accuracy: 0.5430 - val_loss: 2.2355 - val_accuracy: 0.5098
Epoch 59/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.8215 - accuracy: 0.5440 - val_loss: 2.2368 - val_accuracy: 0.5087
Epoch 60/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.8199 - accuracy: 0.5458 - val_loss: 2.2363 - val_accuracy: 0.5106
Epoch 61/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.8137 - accuracy: 0.5461 - val_loss: 2.2380 - val_accuracy: 0.5086
Epoch 62/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.8023 - accuracy: 0.5491 - val_loss: 2.2371 - val_accuracy: 0.5132
Epoch 63/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.7962 - accuracy: 0.5521 - val_loss: 2.2333 - val_accuracy

1905/1905 [==============================] - 4s 2ms/step - loss: 1.5766 - accuracy: 0.6004 - val_loss: 2.2639 - val_accuracy: 0.5366
Epoch 114/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.5728 - accuracy: 0.6004 - val_loss: 2.2671 - val_accuracy: 0.5387
Epoch 115/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.5642 - accuracy: 0.6036 - val_loss: 2.2646 - val_accuracy: 0.5410
Epoch 116/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.5622 - accuracy: 0.6018 - val_loss: 2.2665 - val_accuracy: 0.5409
Epoch 117/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.5630 - accuracy: 0.6048 - val_loss: 2.2686 - val_accuracy: 0.5413
Epoch 118/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.5563 - accuracy: 0.6045 - val_loss: 2.2708 - val_accuracy: 0.5391
Epoch 119/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.5522 - accuracy: 0.6041 - val_loss: 2.2715 - val_ac

Epoch 169/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.4276 - accuracy: 0.6341 - val_loss: 2.3634 - val_accuracy: 0.5483
Epoch 170/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.4305 - accuracy: 0.6340 - val_loss: 2.3588 - val_accuracy: 0.5517
Epoch 171/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.4282 - accuracy: 0.6330 - val_loss: 2.3630 - val_accuracy: 0.5511
Epoch 172/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.4297 - accuracy: 0.6338 - val_loss: 2.3628 - val_accuracy: 0.5489
Epoch 173/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.4216 - accuracy: 0.6359 - val_loss: 2.3741 - val_accuracy: 0.5479
Epoch 174/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.4227 - accuracy: 0.6355 - val_loss: 2.3750 - val_accuracy: 0.5502
Epoch 175/200
1905/1905 [==============================] - 4s 2ms/step - loss: 1.4258 - accuracy: 0.6332 - val_loss: 2